# Universal Sentence Encoder Baseline for V-reg Task


In this notebook, we will walk you through the process of reproducing the Universal Sentence Encoder baseline for the V-reg task.

## Loading Required Modules

We start by loading the needed python libraries.

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import pandas as pd
import tensorflow_text
from tensorflow import keras
from scipy.stats import pearsonr

## Loading Data

Using pandas, we can load and inspect the training, validation, and testing datasets as follows:

In [2]:
df_train = pd.read_csv("../../data/affect-in-tweets/V-reg/2018-Valence-reg-Ar-train.txt", sep="\t")
df_dev = pd.read_csv("../../data/affect-in-tweets/V-reg/2018-Valence-reg-Ar-dev.txt", sep="\t")
df_test = pd.read_csv("../../private_datasets/vreg/vreg_no_labels_v1.0.tsv", sep="\t")

Below we list the 5 first entries in the training data.

In [3]:
df_train.head()

,ID,Tweet,Affect Dimension,Intensity Score
0,2018-Ar-01961,إلىٰ متىٰ الألم يغلب على الفرح,valence,0.097
1,2018-Ar-03289,@Al3mriRami @Holyliviuss كل مافي الأمر أني غاض...,valence,0.219
2,2018-Ar-04349,يحذركم ويخوفكم من نفسه اذا ارتكبتم ذنب او معصي...,valence,0.313
3,2018-Ar-03640,💞 💞 صباحكم سعادة في اليوم المبارك تقبل الله صي...,valence,0.828
4,2018-Ar-01176,@sjalmulla شفته قبل اسبوع ومتشوقه عليه وايد ال...,valence,0.719


And the 5 first entries in the development data.

In [4]:
df_dev.head()

,ID,Tweet,Affect Dimension,Intensity Score
0,2018-Ar-00297,لؤي عرفك من زماان طيب ومحترم وجدع ومحبوب ربنا ...,valence,0.613
1,2018-Ar-03228,مدمن العزلة يخاف الاهتمام الزائد يتوتر لا يحسن...,valence,0.328
2,2018-Ar-00857,تذكر أن بعد الشقاء سعادة وبعد دموعك #إبتسامة,valence,0.625
3,2018-Ar-02764,ماف واحد متزوج اسأله عن الزواج الا يسب ويلعن و...,valence,0.422
4,2018-Ar-00582,٢٥ للاسف ما بعرفك بس باين انك حد منيح ودمك خفي...,valence,0.547


And last but not least, the first 5 entries in the test data.

In [5]:
df_test.head()

,ID,Tweet,Affect Dimension,Intensity Score
0,ID-923,للاسف اتى علينا زمن اصبح بعض الآباء ليس حضناً ...,valence,NONE
1,ID-280,ايه الفرص اللي بتضيع من البرازيل دي حراام بجد,valence,NONE
2,ID-406,جات لى ريادة أطفال .. ف الاسبوع السادس,valence,NONE
3,ID-423,الحمد لله انه ما في خاصيه بتبين كم مره حضرت ال...,valence,NONE
4,ID-965,اب همي وهم بي أحبابي همهم ما بهم وهمي ما بي ...,valence,NONE


## Model Preparation

We start by setting the randomisation seed:

In [6]:
tf.random.set_seed(123)

Next we load the Universal Sentence Encoder (WARNING: This will download and cache a huge model of around 1 GB in size)

In [7]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

Then we define the input and output to the model:

In [8]:
sentence = keras.Input(shape=512, name='sentence')
label = keras.Input(shape=(1,), name='label')

This is followed by defining the structure of the network:

In [9]:
logits = keras.layers.Dense(512, activation=tf.nn.sigmoid)(sentence)
logits = keras.layers.Dense(512, activation=tf.nn.sigmoid)(logits)
logits = keras.layers.Dense(1, activation=tf.nn.sigmoid)(logits)

Then we construct and compile the model:

In [10]:
model = keras.Model(sentence, outputs=logits)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

## Model Training

First we perpare the inputs and outputs to be fed to the model during training:

In [11]:
X_train = embed(df_train["Tweet"])
Y_train = df_train["Intensity Score"]
X_dev = embed(df_dev["Tweet"])
Y_dev = df_dev["Intensity Score"]

Next we fit the data:

In [12]:
model.fit(X_train,
          Y_train,
          epochs=10,
          batch_size=32,
          validation_data = (X_dev, Y_dev))

Epoch 1/10
30/30 [==============================] - 2s 34ms/step - loss: 0.0818 - mae: 0.2390 - val_loss: 0.0691 - val_mae: 0.2257
Epoch 2/10
30/30 [==============================] - 0s 8ms/step - loss: 0.0544 - mae: 0.1990 - val_loss: 0.0361 - val_mae: 0.1574
Epoch 3/10
30/30 [==============================] - 0s 8ms/step - loss: 0.0510 - mae: 0.1875 - val_loss: 0.0509 - val_mae: 0.1920
Epoch 4/10
30/30 [==============================] - 0s 9ms/step - loss: 0.0424 - mae: 0.1701 - val_loss: 0.0660 - val_mae: 0.2209
Epoch 5/10
30/30 [==============================] - 0s 8ms/step - loss: 0.0402 - mae: 0.1620 - val_loss: 0.0289 - val_mae: 0.1294
Epoch 6/10
30/30 [==============================] - 0s 8ms/step - loss: 0.0282 - mae: 0.1346 - val_loss: 0.0312 - val_mae: 0.1439
Epoch 7/10
30/30 [==============================] - 0s 8ms/step - loss: 0.0278 - mae: 0.1317 - val_loss: 0.0251 - val_mae: 0.1266
Epoch 8/10
30/30 [==============================] - 0s 7ms/step - loss: 0.0259 - mae: 0.1

We calculate the Pearson correlation coefficient for the development set as follows:

In [13]:
pearsonr(Y_dev, model.predict(X_dev).reshape(-1))

(0.6589736134728249, 1.5493966539516294e-18)

## Submission Preperation

We perpare the features for each testset instance as follows:

In [14]:
X_test = embed(df_test["Tweet"])

Then we predict the labels for each:

In [15]:
predictions = model.predict(X_test)

We perpare the predictions as a pandas dataframe.

In [16]:
df_preds = pd.DataFrame(data=predictions, columns=["prediction"], index=df_test["ID"])
df_preds.reset_index(inplace=True)

In the final step, we save the predictions as required by the competition guidelines.

In [17]:
if not os.path.exists("predictions"):
    os.mkdir("predictions")
df_preds.to_csv("./predictions/v_reg.tsv", index=False, sep="\t")